In [2]:
!pip install editdistance
!pip install sentencepiece transformers==4.33 datasets sacremoses sacrebleu  -q
!pip install editdistance

import locale
import pandas as pd
import numpy as np
import re
import sys
import typing as tp
import unicodedata
import gc
import random
import torch
import sacrebleu
import editdistance
from sklearn.model_selection import train_test_split
from transformers import NllbTokenizer, AutoModelForSeq2SeqLM, AutoConfig, get_constant_schedule_with_warmup
from tqdm.auto import tqdm, trange
from transformers.optimization import Adafactor
from sacremoses import MosesPunctNormalizer

pd.options.display.max_colwidth = 100

In [3]:
trans_df = pd.read_csv('trans_df.csv')

trans_df.drop(columns = 'Unnamed: 0', inplace = True)
trans_df = trans_df[trans_df.ru.notna()]
print(trans_df.isnull().sum())

print(trans_df.split.value_counts())

df_train = trans_df[trans_df.split=='train'].copy()
df_dev = trans_df[trans_df.split=='dev'].copy()
df_test = trans_df[trans_df.split=='test'].copy()

ru       0
tyv      0
split    0
dtype: int64
split
train    108183
test       5000
dev        4999
Name: count, dtype: int64


In [6]:
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.55k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/564 [00:00<?, ?B/s]

In [12]:
def word_tokenize(text):
    return re.findall('(\w+|[^\w\s])', text)

texts_with_unk = [text for text in tqdm(trans_df.tyv) if tokenizer.unk_token_id in tokenizer(text).input_ids]
print(len(texts_with_unk))

  0%|          | 0/118182 [00:00<?, ?it/s]

3646


In [13]:
mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [ ]:
texts_with_unk_normed = [text for text in tqdm(texts_with_unk) if tokenizer.unk_token_id in tokenizer(preproc(text)).input_ids]
print(len(texts_with_unk_normed))

  0%|          | 0/3646 [00:00<?, ?it/s]

18


In [6]:
def fix_tokenizer(tokenizer, new_lang='tyv_Cyrl'):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [ ]:
fix_tokenizer(tokenizer)

In [17]:
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'tyv_Cyrl', '<mask>'])) # [256202, 256203, 256204]

[256202, 256203, 256204]


In [18]:
added_token_id = tokenizer.convert_tokens_to_ids('tyv_Cyrl')
similar_lang_id = tokenizer.convert_tokens_to_ids('kir_Cyrl')
print(added_token_id, similar_lang_id)

256203 256095


In [35]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write)

In [5]:
MODEL_URL = 'Nacoor/mbart-v1'
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_URL)
tokenizer = NllbTokenizer.from_pretrained(MODEL_URL, force_download=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/3.56k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
fix_tokenizer(tokenizer)

In [9]:
def translate(
    text,
    model,
    tokenizer,
    src_lang='rus_Cyrl',
    tgt_lang='tyv_Cyrl',
    max_length='auto',
    num_beams=4,
    no_repeat_ngram_size=4,
    n_out=None,
    **kwargs
):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    if max_length == 'auto':
        max_length = int(32 + 2.0 * encoded.input_ids.shape[1])
    model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out

In [11]:
translate("когда моя мама приехала домой, мои друзья уже начали есть", model=model, tokenizer=tokenizer)

'кажан авам аалынга чедип келгенде, мээң эштерим чип эгелээн'

In [12]:
translate("сколько тебе лет исполнится в следующем году?", model=model, tokenizer=tokenizer)

'сен келир чылын каш харлыг апаар сен?'

In [16]:
def translate(text, src_lang='rus_Cyrl', tgt_lang='eng_Latn', a=32, b=3, max_input_length=1024, num_beams=4, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [13]:
from datasets import load_dataset
dataset = load_dataset("AigizK/tatar-russian-parallel-corpora")

Generating train split:   0%|          | 0/161831 [00:00<?, ? examples/s]

In [14]:
additional_df = pd.DataFrame(dataset['train'])
additional_df = additional_df[-20000:]
additional_df

,tat,rus
141831,— Беркем дә кабул ителми.,– Не принимать.
141832,"Француз әллә чынлап, әллә юри йоклый, башын кресло аркасына салган, тезе өстендә яткан тирләгән ...","Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш..."
141833,"Алексей Александрович урыныннан торды да, бик сак барырга теләсә дә, өстәлгә абына-абына, францу...","Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в..."
141834,"Сте¬пан Аркадьич та урыныннан торды һәм, күзләрен зур итеп ачып, әгәр йоклый икән, үзен уятырга ...","Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ..."
141835,"Боларның барысы да төш түгел, өн иде.",Все это было наяву.
...,...,...
161826,Петро кулын Михаилның иңбашына салды һәм башы белән далага ымлап күрсәтте.,Петро притронулся рукой к плечу Михаила и кивнул на степь.
161827,"Ул өстенә чибәр генә кара костюм кигән, юлдан ерак түгел куе иген арасыннан баручы картрак бер к...","Он давно уже приметил пожилого, одетого в добротный темный костюм человека, шагающего в густых х..."
161828,Сөрелгән җиргә җиткәч ул юлга борылды.,"Дойдя до вспаханной земли, человек свернул на дорогу."
161829,"Ул сугышчыларның сак карашларын сизеп алды да, йокыдан уянгандай булып, тиз генә сөйли башлады:","Он встретился с настороженными взглядами бойцов и, будто очнувшись, быстро проговорил:"


In [18]:
model.cuda();

In [20]:
additional_df['tyv'] = [translate(t, 'rus_Cyrl', 'tyv_Cyrl')[0] for t in tqdm(additional_df.rus)]

  0%|          | 0/20000 [00:00<?, ?it/s]

In [21]:
additional_df

,tat,rus,tyv
141831,— Беркем дә кабул ителми.,– Не принимать.,хүлээп албас.
141832,"Француз әллә чынлап, әллә юри йоклый, башын кресло аркасына салган, тезе өстендә яткан тирләгән ...","Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш...","Француз кижи удуп азы удуп турар бооп баажыланып, бажын креслонуң ооргазынче чыпшыр салгаш, диск..."
141833,"Алексей Александрович урыныннан торды да, бик сак барырга теләсә дә, өстәлгә абына-абына, францу...","Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в...","Алексей Александрвич туруп келгеш, оваарымчалыг болуксаан, ынчалза-даа столга илбектеп алгаш, чо..."
141834,"Сте¬пан Аркадьич та урыныннан торды һәм, күзләрен зур итеп ачып, әгәр йоклый икән, үзен уятырга ...","Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ...","Аркадьич хам туруп келгеш, карактарын хере көрүп, удуп чыдырда оттурар дээн күзел-биле бирээзинч..."
141835,"Боларның барысы да төш түгел, өн иде.",Все это было наяву.,Ол бүгү ылап-ла болган.
...,...,...,...
161826,Петро кулын Михаилның иңбашына салды һәм башы белән далага ымлап күрсәтте.,Петро притронулся рукой к плечу Михаила и кивнул на степь.,"Петр холу-биле ол кижиниң эктинге дээпкеш, ховуже бажын согаш кылды."
161827,"Ул өстенә чибәр генә кара костюм кигән, юлдан ерак түгел куе иген арасыннан баручы картрак бер к...","Он давно уже приметил пожилого, одетого в добротный темный костюм человека, шагающего в густых х...",Оруктан ырак эвеске сырый хлебчигештерлиг кылаштап чоруур кара-кара костюмнуг кырган кижини ол ш...
161828,Сөрелгән җиргә җиткәч ул юлга борылды.,"Дойдя до вспаханной земли, человек свернул на дорогу.","Чеде берген черге чеде бергеш, ол кижи орукче ээпти."
161829,"Ул сугышчыларның сак карашларын сизеп алды да, йокыдан уянгандай булып, тиз генә сөйли башлады:","Он встретился с настороженными взглядами бойцов и, будто очнувшись, быстро проговорил:","Дайынчыларның кичээнгейлиг көрүжү-биле ол таваржы бергеш, оттуп келген дег, дүрген-не мынча дээн:"


In [23]:
additional_df.rename(columns = {'rus': 'ru'}, inplace = True)

AttributeError: 'NoneType' object has no attribute 'drop'

In [27]:
additional_trans_df = additional_df.drop(columns = 'tat')
additional_trans_df.reset_index(drop = True, inplace = True)
additional_trans_df

,ru,tyv
0,– Не принимать.,хүлээп албас.
1,"Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш...","Француз кижи удуп азы удуп турар бооп баажыланып, бажын креслонуң ооргазынче чыпшыр салгаш, диск..."
2,"Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в...","Алексей Александрвич туруп келгеш, оваарымчалыг болуксаан, ынчалза-даа столга илбектеп алгаш, чо..."
3,"Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ...","Аркадьич хам туруп келгеш, карактарын хере көрүп, удуп чыдырда оттурар дээн күзел-биле бирээзинч..."
4,Все это было наяву.,Ол бүгү ылап-ла болган.
...,...,...
19995,Петро притронулся рукой к плечу Михаила и кивнул на степь.,"Петр холу-биле ол кижиниң эктинге дээпкеш, ховуже бажын согаш кылды."
19996,"Он давно уже приметил пожилого, одетого в добротный темный костюм человека, шагающего в густых х...",Оруктан ырак эвеске сырый хлебчигештерлиг кылаштап чоруур кара-кара костюмнуг кырган кижини ол ш...
19997,"Дойдя до вспаханной земли, человек свернул на дорогу.","Чеде берген черге чеде бергеш, ол кижи орукче ээпти."
19998,"Он встретился с настороженными взглядами бойцов и, будто очнувшись, быстро проговорил:","Дайынчыларның кичээнгейлиг көрүжү-биле ол таваржы бергеш, оттуп келген дег, дүрген-не мынча дээн:"


In [28]:
final_trans_df = pd.concat([additional_trans_df, trans_df], axis = 0)
final_trans_df

,ru,tyv,split
0,– Не принимать.,хүлээп албас.,NaN
1,"Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш...","Француз кижи удуп азы удуп турар бооп баажыланып, бажын креслонуң ооргазынче чыпшыр салгаш, диск...",NaN
2,"Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в...","Алексей Александрвич туруп келгеш, оваарымчалыг болуксаан, ынчалза-даа столга илбектеп алгаш, чо...",NaN
3,"Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ...","Аркадьич хам туруп келгеш, карактарын хере көрүп, удуп чыдырда оттурар дээн күзел-биле бирээзинч...",NaN
4,Все это было наяву.,Ол бүгү ылап-ла болган.,NaN
...,...,...,...
118177,Черновицкая область,Черновиц можу,dev
118178,физический,күш,train
118179,храбрый баран,дидим кошкар,train
118180,Вы вправе верить или не верить учению Библии.,"Библияның өөредиинге бүзүрээрин, бүзүревезин боттарыңар билир силер.",train


In [29]:
final_trans_df.reset_index(drop = True, inplace = True)
final_trans_df.fillna('train', inplace = True)
final_trans_df

,ru,tyv,split
0,– Не принимать.,хүлээп албас.,train
1,"Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш...","Француз кижи удуп азы удуп турар бооп баажыланып, бажын креслонуң ооргазынче чыпшыр салгаш, диск...",train
2,"Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в...","Алексей Александрвич туруп келгеш, оваарымчалыг болуксаан, ынчалза-даа столга илбектеп алгаш, чо...",train
3,"Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ...","Аркадьич хам туруп келгеш, карактарын хере көрүп, удуп чыдырда оттурар дээн күзел-биле бирээзинч...",train
4,Все это было наяву.,Ол бүгү ылап-ла болган.,train
...,...,...,...
138177,Черновицкая область,Черновиц можу,dev
138178,физический,күш,train
138179,храбрый баран,дидим кошкар,train
138180,Вы вправе верить или не верить учению Библии.,"Библияның өөредиинге бүзүрээрин, бүзүревезин боттарыңар билир силер.",train


In [30]:
final_trans_df.to_csv('back_translated_trans_df.csv')

In [33]:
from google.colab import files

files.download('back_translated_trans_df.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
final_trans_df = pd.read_csv('back_translated_trans_df.csv')
final_trans_df

,Unnamed: 0,ru,tyv,split
0,0,– Не принимать.,хүлээп албас.,train
1,1,"Француз спал или притворялся, что спит, прислонив голову к спинке кресла, и потною рукой, лежавш...","Француз кижи удуп азы удуп турар бооп баажыланып, бажын креслонуң ооргазынче чыпшыр салгаш, диск...",train
2,2,"Алексей Александрович встал, хотел осторожно, но, зацепив за стол, подошел и положил свою руку в...","Алексей Александрвич туруп келгеш, оваарымчалыг болуксаан, ынчалза-даа столга илбектеп алгаш, чо...",train
3,3,"Степан Аркадьич встал тоже и, широко отворяя глаза, желая разбудить себя, если он спит, смотрел ...","Аркадьич хам туруп келгеш, карактарын хере көрүп, удуп чыдырда оттурар дээн күзел-биле бирээзинч...",train
4,4,Все это было наяву.,Ол бүгү ылап-ла болган.,train
...,...,...,...,...
138177,138177,Черновицкая область,Черновиц можу,dev
138178,138178,физический,күш,train
138179,138179,храбрый баран,дидим кошкар,train
138180,138180,Вы вправе верить или не верить учению Библии.,"Библияның өөредиинге бүзүрээрин, бүзүревезин боттарыңар билир силер.",train


In [4]:
final_trans_df = final_trans_df[final_trans_df.ru.notna()]
final_trans_df.isnull().sum()

Unnamed: 0    0
ru            0
tyv           0
split         0
dtype: int64

In [5]:
final_trans_df.split.value_counts()

split
train    128183
test       5000
dev        4999
Name: count, dtype: int64

In [6]:
df_train = final_trans_df[final_trans_df.split=='train'].copy()
df_dev = final_trans_df[final_trans_df.split=='dev'].copy()
df_test = final_trans_df[final_trans_df.split=='test'].copy()

In [7]:
tokenizer = NllbTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
def word_tokenize(text):
    return re.findall('(\w+|[^\w\s])', text)

In [9]:
smpl = df_train.sample(10000, random_state=1)

smpl['rus_toks'] = smpl.ru.apply(tokenizer.tokenize)
smpl['tyv_toks'] = smpl.tyv.apply(tokenizer.tokenize)

smpl['rus_words'] = smpl.ru.apply(word_tokenize)
smpl['tyv_words'] = smpl.tyv.apply(word_tokenize)

stats = smpl[['rus_toks', 'tyv_toks', 'rus_words', 'tyv_words']].applymap(len).describe()
print(stats.rus_toks['mean'] / stats.rus_words['mean'])
print(stats.tyv_toks['mean'] / stats.tyv_words['mean'])

1.7461067366579177
2.2146619071305262


In [11]:
texts_with_unk = [text for text in tqdm(final_trans_df.tyv) if tokenizer.unk_token_id in tokenizer(text).input_ids]
print(len(texts_with_unk))

  0%|          | 0/138182 [00:00<?, ?it/s]

3646


In [12]:
mpn = MosesPunctNormalizer(lang="en")
mpn.substitutions = [
    (re.compile(r), sub) for r, sub in mpn.substitutions
]


def get_non_printing_char_replacer(replace_by: str = " ") -> tp.Callable[[str], str]:
    non_printable_map = {
        ord(c): replace_by
        for c in (chr(i) for i in range(sys.maxunicode + 1))
        if unicodedata.category(c) in {"C", "Cc", "Cf", "Cs", "Co", "Cn"}
    }

    def replace_non_printing_char(line) -> str:
        return line.translate(non_printable_map)

    return replace_non_printing_char

replace_nonprint = get_non_printing_char_replacer(" ")

def preproc(text):
    clean = mpn.normalize(text)
    clean = replace_nonprint(clean)
    clean = unicodedata.normalize("NFKC", clean)
    return clean

In [13]:
texts_with_unk_normed = [text for text in tqdm(texts_with_unk) if tokenizer.unk_token_id in tokenizer(preproc(text)).input_ids]
print(len(texts_with_unk_normed))

  0%|          | 0/3646 [00:00<?, ?it/s]

18


In [14]:
print(len(tokenizer))
print(tokenizer.convert_ids_to_tokens([256202, 256203]))

256204
['zul_Latn', '<mask>']


In [15]:
def fix_tokenizer(tokenizer, new_lang='tyv_Cyrl'):
    """
    Add a new language token to the tokenizer vocabulary
    (this should be done each time after its initialization)
    """
    old_len = len(tokenizer) - int(new_lang in tokenizer.added_tokens_encoder)
    tokenizer.lang_code_to_id[new_lang] = old_len-1
    tokenizer.id_to_lang_code[old_len-1] = new_lang
    # always move "mask" to the last position
    tokenizer.fairseq_tokens_to_ids["<mask>"] = len(tokenizer.sp_model) + len(tokenizer.lang_code_to_id) + tokenizer.fairseq_offset

    tokenizer.fairseq_tokens_to_ids.update(tokenizer.lang_code_to_id)
    tokenizer.fairseq_ids_to_tokens = {v: k for k, v in tokenizer.fairseq_tokens_to_ids.items()}
    if new_lang not in tokenizer._additional_special_tokens:
        tokenizer._additional_special_tokens.append(new_lang)
    # clear the added token encoder; otherwise a new token may end up there by mistake
    tokenizer.added_tokens_encoder = {}
    tokenizer.added_tokens_decoder = {}

In [16]:
fix_tokenizer(tokenizer)

In [17]:
print(tokenizer.convert_ids_to_tokens([256202, 256203, 256204])) # ['zul_Latn', 'tyv_Cyrl', '<mask>']
print(tokenizer.convert_tokens_to_ids(['zul_Latn', 'tyv_Cyrl', '<mask>'])) # [256202, 256203, 256204]
# this is consistent now, wow!

['zul_Latn', 'tyv_Cyrl', '<mask>']
[256202, 256203, 256204]


In [18]:
added_token_id = tokenizer.convert_tokens_to_ids('tyv_Cyrl')
similar_lang_id = tokenizer.convert_tokens_to_ids('kir_Cyrl')
print(added_token_id, similar_lang_id)

256203 256095


In [19]:
model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model.resize_token_embeddings(len(tokenizer))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 256205. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


Embedding(256205, 1024)

In [20]:
model.model.shared.weight.data[added_token_id+1] = model.model.shared.weight.data[added_token_id]
model.model.shared.weight.data[added_token_id] = model.model.shared.weight.data[similar_lang_id]

In [21]:
def cleanup():
    """Try to free GPU memory"""
    gc.collect()
    torch.cuda.empty_cache()

cleanup()

In [22]:
model.cuda();

In [23]:
optimizer = Adafactor(
    [p for p in model.parameters() if p.requires_grad],
    scale_parameter=False,
    relative_step=False,
    lr=1e-4,
    clip_threshold=1.0,
    weight_decay=1e-3,
)

In [24]:
batch_size = 8  # 32 already doesn't fit well to 15GB of GPU memory
max_length = 128
warmup_steps = 1_000
training_steps = 57000

In [25]:
losses = []
scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps)

In [26]:
LANGS = [('ru', 'rus_Cyrl'), ('tyv', 'tyv_Cyrl')]

def get_batch_pairs(batch_size, data=df_train):
    (l1, long1), (l2, long2) = random.sample(LANGS, 2)
    xx, yy = [], []
    for _ in range(batch_size):
        item = data.iloc[random.randint(0, len(data)-1)]
        xx.append(preproc(item[l1]))
        yy.append(preproc(item[l2]))
    return xx, yy, long1, long2

print(get_batch_pairs(1))
# (['чеди'], ['семь'], 'tyv_Cyrl', 'rus_Cyrl')

(['Бо болуушкун чажыт утка-биле долдунган... - деп, Бэктэрниң өлүмү дээш харыысаалганы боттарынга алган Тэмучин биле Хасар дугайында бодалдарын чөвүлекчилер делгереңгейи-биле илередип турганнар.'], ['"Этот поступок полон скрытого смысла, - пространно выразились старейшины о Тэмучине с Хасаром, взявшим на себя ответственность за смерть Бэктэра.'], 'tyv_Cyrl', 'rus_Cyrl')


In [28]:
model.train()
x, y, loss = None, None, None
cleanup()

tq = trange(len(losses), training_steps)
for i in tq:
    xx, yy, lang1, lang2 = get_batch_pairs(batch_size)
    try:
        tokenizer.src_lang = lang1
        x = tokenizer(xx, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        tokenizer.src_lang = lang2
        y = tokenizer(yy, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(model.device)
        y.input_ids[y.input_ids == tokenizer.pad_token_id] = -100

        loss = model(**x, labels=y.input_ids).loss
        loss.backward()
        losses.append(loss.item())

        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        scheduler.step()

    except RuntimeError as e:
        optimizer.zero_grad(set_to_none=True)
        x, y, loss = None, None, None
        cleanup()
        print('error', max(len(s) for s in xx + yy), e)
        continue

    if i % 1000 == 0:
        print(i, np.mean(losses[-1000:]))

  0%|          | 0/57000 [00:00<?, ?it/s]

0 4.064620494842529
1000 3.5855690891742706
2000 2.6898501447439194
3000 2.346245167732239
4000 2.1832066876888274
5000 2.042162572801113
6000 1.946362805724144
7000 1.880080339550972
8000 1.7923041107654571
9000 1.7584814242124558
10000 1.6804028256237507
11000 1.6451089248657227
12000 1.6512788317203522
13000 1.5781691719293593
14000 1.5442592724561692
15000 1.5276449618637562
16000 1.4900237018465996
17000 1.4609340980648995
18000 1.4308358041644096
19000 1.4145816402435303
20000 1.3918941928744315
21000 1.3745912779569627
22000 1.3547322038114071
23000 1.32668189483881
24000 1.2988270655274392
25000 1.2929947068691254
26000 1.2971106440722941
27000 1.2816256609857082
28000 1.2836293933093548
29000 1.2478779553472996
30000 1.2147990318834783
31000 1.23210057502985
32000 1.2074449253082276
33000 1.1867650341689586
34000 1.1761490099132061
35000 1.1673222172409297
36000 1.1622699163854122
37000 1.1463221500515939
38000 1.1332960777282715
39000 1.1386249847710133
40000 1.11354799818992

In [31]:
def translate(text, src_lang='rus_Cyrl', tgt_lang='eng_Latn', a=32, b=3, max_input_length=1024, num_beams=4, **kwargs):
    tokenizer.src_lang = src_lang
    tokenizer.tgt_lang = tgt_lang
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=max_input_length)
    result = model.generate(
        **inputs.to(model.device),
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(tgt_lang),
        max_new_tokens=int(a + b * inputs.input_ids.shape[1]),
        num_beams=num_beams,
        **kwargs
    )
    return tokenizer.batch_decode(result, skip_special_tokens=True)

In [32]:
df_dev['rus_translated'] = [translate(t, 'tyv_Cyrl', 'rus_Cyrl')[0] for t in tqdm(df_dev.tyv)]
df_dev['tyv_translated'] = [translate(t, 'rus_Cyrl', 'tyv_Cyrl')[0] for t in tqdm(df_dev.ru)]

  0%|          | 0/4999 [00:00<?, ?it/s]

  0%|          | 0/4999 [00:00<?, ?it/s]

In [33]:
import sacrebleu
bleu_calc = sacrebleu.BLEU()
chrf_calc = sacrebleu.CHRF(word_order=2)  # this metric is called ChrF++

In [34]:
print(bleu_calc.corpus_score(df_dev['rus_translated'].tolist(), [df_dev['ru'].tolist()]))
print(chrf_calc.corpus_score(df_dev['rus_translated'].tolist(), [df_dev['ru'].tolist()]))
print(bleu_calc.corpus_score(df_dev['tyv_translated'].tolist(), [df_dev['tyv'].tolist()]))
print(chrf_calc.corpus_score(df_dev['tyv_translated'].tolist(), [df_dev['tyv'].tolist()]))

BLEU = 17.61 46.5/23.4/13.2/7.7 (BP = 0.966 ratio = 0.966 hyp_len = 22426 ref_len = 23207)
chrF2++ = 42.24
BLEU = 18.03 47.1/23.7/13.3/7.8 (BP = 0.977 ratio = 0.977 hyp_len = 22652 ref_len = 23188)
chrF2++ = 45.83


In [36]:
upload_repo = "Nacoor/nllb600_back_translation"
tokenizer.push_to_hub(upload_repo)
model.push_to_hub(upload_repo)

sentencepiece.bpe.model:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.46G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Nacoor/nllb600_back_translation/commit/965f6d3cb90b4383f127654fd6a291772e8e80a3', commit_message='Upload M2M100ForConditionalGeneration', commit_description='', oid='965f6d3cb90b4383f127654fd6a291772e8e80a3', pr_url=None, pr_revision=None, pr_num=None)

In [37]:
def translate(
    text,
    model,
    tokenizer,
    src_lang='rus_Cyrl',
    tgt_lang='tyv_Cyrl',
    max_length='auto',
    num_beams=4,
    no_repeat_ngram_size=4,
    n_out=None,
    **kwargs
):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    if max_length == 'auto':
        max_length = int(32 + 2.0 * encoded.input_ids.shape[1])
    model.eval()
    generated_tokens = model.generate(
        **encoded.to(model.device),
        forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang],
        max_length=max_length,
        num_beams=num_beams,
        no_repeat_ngram_size=no_repeat_ngram_size,
        num_return_sequences=n_out or 1,
        **kwargs
    )
    out = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    if isinstance(text, str) and n_out is None:
        return out[0]
    return out

In [38]:
translate("когда моя мама приехала домой, мои друзья уже начали есть", model=model, tokenizer=tokenizer)

'авам чанып келгенде, эш-өөрүм чемненип эгелээн'

In [39]:
translate("сколько тебе лет исполнится в следующем году?", model=model, tokenizer=tokenizer)

'келир чылын каш харлыг болур сен?'